*please execute the cell below before starting the tutorial by selecting it and pressing Ctrl+Enter*

In [ ]:
from aiida.orm import Dict, load_node
from aiida.engine import submit
from aiida import load_profile

load_profile()

# Input generator code

On Monday you learned about the simplified input file for the FLEUR code. The inpgen code generates an `inp.xml` input-file for FLEUR using the simplified input. In AiiDA, a `Code` object representing input generator has to have a similar behaviour: it has to generate a `FleurinpData` from a list of simplified parameters. The code class wrapping inpgen code is called `FleurinputgenCalculation`. In this tutorial we are going to learn how to use it.

First things first - let us get familiar with inputs of the `FleurinputgenCalculation`.

## Inputs of the input calculation

`FleurinputgenCalculation` has four main input nodes listed in the table below:

|name|type|description| required |
|:---:|:---:|:---------:|:---:|
|code | Code | Inpgen code| yes |
|structure | StructureData | Structure data node| yes |
|parameters | Dict | FLAPW parameters| no |
|settings | Dict | special settings | no |
|metadata.options | Dict | computational resources | yes |

**code** is a `Code` representing inpgen inself in the database. 

Simplified input file is represented as a pair of input nodes: **structure** and **parameters**. **structure** is a `StructureData` node to be used in the simplified input file (`&lattice`). **parameters** is a nested dictionary containing nested name: value pairs and representing all other parameters of the simplified input file. For instance, if one wants to add a line

    &kpt
      div1=4   div2=5   div3=6 /

to the simplified input file, it is necessary to append

    'kpt': {
        'div1': 4,
        'div2' : 5,
        'div3' : 6
        }

to the **parameters** dictionary.

In some cases the simplified input file does not have names for nested values. In this case you can refer to the list of all supported keys:

    'input': ['film', 'cartesian', 'cal_symm', 'checkinp', 'symor', 'oldfleur']
    
    'atom': ['id', 'z', 'rmt', 'dx', 'jri', 'lmax', 'lnonsph', 'ncst', 'econfig', 'bmu', 'lo', 'element', 'name']
             
    'comp': ['jspins', 'frcor', 'ctail', 'kcrel', 'gmax', 'gmaxxc', 'kmax']
    
    'exco': ['xctyp', 'relxc'],
    
    'film': ['dvac', 'dtild'],
    
    'soc': ['theta', 'phi'],
    
    'qss': ['x', 'y', 'z'],
    
    'kpt': ['nkpt', 'kpts', 'div1', 'div2', 'div3','tkb', 'tria'],
    
    'title': {}
              
For example, you can find that SOC ``theta`` and ``phi`` values can be set appending

    'soc': {
        'theta': 0.7,
        'phi' : 0.26
        }

to the **parameters** dictionary, resulting in:

    &soc
      0.7 0.26 /
      

**settings** is used to modify a list of files to be copied to/from/between calculation folders and to add command options to the executable. We will not cover this in the tutorial.


## Inpgen code submission

### Inputs preparation 

To submit an inpgen calculation, we need to import `FleurinputgenCalculation` class

In [ ]:
from aiida_fleur.calculation.fleurinputgen import FleurinputgenCalculation

or use the `CalculationFactory`:

In [ ]:
from aiida.plugins import CalculationFactory
FleurinputgenCalculation = CalculationFactory('fleur.inpgen')

and prepare all the inputs. Configured and ready-to-use inpgen code node is already stored in your database under PK=8. You can simply load it from the database:

In [ ]:
inpgen_code = load_node(7)

We are going to use a structure that you created in tutorial 1. Please, find a PK of the Fe monolayer structure and load it via the `load_node()` command:

In [ ]:
# you need to modify this - replace FE_PK
structure_FeML = load_node(FE_PK)

Next, we need to set up the **parameters** dictionary. For now we are going to specify k-mesh and kmax only:

In [ ]:
parameters = Dict(dict={
    'comp': {
        'kmax': 3.85,
        },
    'kpt': {
        'div1': 4,
        'div2' : 4,
        'div3' : 4
        }})

Finally, we need to specify computational resources for the task. Since inpgen usually takes a few seconds to finish, we always run it in serial, passing False to 'withmpi' and setting 1 MPI task.

In [ ]:
options = {'resources' : {"num_machines": 1, "num_mpiprocs_per_machine" : 1}, 'withmpi' : False}

### Assembling inputs in a single dictionary

All the input modes can be passed separately to the `FleurinpgenCalculation`. However, it much more convenient to assemble them in a single dictionary first, using `get_builder()` method:

In [ ]:
inputs = FleurinputgenCalculation.get_builder()
inputs.parameters = parameters
inputs.code = inpgen_code
inputs.structure = structure_FeML
inputs.metadata.options = options  # note options not in inputs but inputs.metadata

### Submission


To submit the input generator, run `submit` function passing the input dictionary:

In [ ]:
inpgen_process = submit(FleurinputgenCalculation, **inputs)
print('The PK of submitted inpgen job is {}'.format(inpgen_process))

`inpgen_pk` contains a PK of the input generator calculation node stored in the database. Later we can access all the information related to this run via this PK.

## Results analysis

After inpgen code is submitted, you can check the status of all processes submitted not more than 24 hours ago:

In [ ]:
!verdi process list -a -p 1

You can also check the status of inpgen calculation directly. Try running next cell and examine the output:

In [ ]:
# you need to modify this: insert an integer instead of INPGEN_PK
!verdi process status INPGEN_PK

Since input generator takes a few seconds to finish, you most probably will not be able to catch it in any other state except 'Finished'. Now let us explore the output nodes. `Fleurinpgencalculation` generates three output nodes:

|name| type | comment|
|:--:|:----:|:----------:|
|fleurinpData|FleurinpData| represents `inp.xml`|
|remote_folder| FolderData| represents calculation folder |
|retrieved| FolderData | represents retrieved folder |

You can get a list of their PKs running:

In [ ]:
# you need to modify this: insert an integer instead of INPGEN_PK
!verdi process show INPGEN_PK

Now you can load any of the outputs via `load_node()` function. In addition, all the outputs can be found in `calc_node.outputs`:

In [ ]:
# you need to modify this: insert an integer instead of INPGEN_PK
inpgen_calc_node = load_node(INPGEN_PK)
output_fleurinp = inpgen_calc_node.outputs.fleurinpData
print('PK of the output FleurinpData is {}'.format(output_fleurinp.pk))

Please, remember the `FleurinpData` PK, we are going to use this in our next tutorial.

# Scripting tasks

Jupyter notebooks are convinient way to learn and run some jobs. However, I personally prefer using and running python scripts in a terminal for my work. As a final task of each tutorial, beggining from this one, you will be asked to construct a small python script and run it in a terminal.

First, you need to start a new terminal and type in:

     $workon aiida
     
To execute a python-script, run

     $(aiida) verdi run NAME_OF_THE_SCRIPT.py

#### 1.  Run an inpgen calculation for Si

Write a script that submits `FleurinpgenCalculation` for a Si structure given in the attached `Si.cif` file. Try to set up **parameters** dictionary to use parameters:

| name | value | comment |
|:----:|:-----:|:-------:|
| atom -> rmt  | 2.23  | muffin-tin radius |
| comp -> kmax | 3.84  | plane wave cut-off|
| kpt -> div1  | 2 | number of kpts along x|
| kpt -> div2  | 2 | number of kpts along y|
| kpt -> div3  | 2 | number of kpts along z|



Try finding out the PK of the output `FleurinpData` and remember it - you will need it in the next tutorial!

#### 2.  Run an inpgen calculation for film structures

Write a script that submits `FleurinpgenCalculation` for a fcc Fe, Ni and Co mololayer film (110) structures with a perioud length = 4.16 Å. Try to set up **parameters** dictionary to use parameters:

| name | value | comment |
|:----:|:-----:|:-------:|
| comp -> kmax | 3.75  | plane wave cut-off|
| kpt -> div1  | 4 | number of kpts along x|
| kpt -> div2  | 4 | number of kpts along y|
| kpt -> div3  | 1 | number of kpts along z|

Also make sure that you remember all structures and `FleurinpData` PKs - we will use it later!

*Hint*: Use `StructureData` methods.
fcc (110) monolayer can be represented as a cell having a single atom at (0, 0, 0):

    cell = [[0.7071068*a, 0.0, 0.0],
            [0.0, 1.0*a, 0.0],
            [0.0, 0.0, 0.7071068*a]]

where a is a period length. You also need to set up periodic boundary conditions via

    structure.pbc = (True, True, False)

Try finding out PKs of the output `FleurinpData` and remember it - you will need it in the next tutorial!